In [1]:
## Installing requirements to be able to run sql queries in Jupyter notebook.

import pandas as pd 
import csv
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

subscriptions= pd.read_csv("subscriptions.csv", engine="python", error_bad_lines=False)
charges= pd.read_csv("charges.csv", engine="python", error_bad_lines=False)
customers= pd.read_csv("customers.csv", engine="python", error_bad_lines=False)


## Setting up PostgreSQL connection.


import sqlalchemy as sa
import credentials_fd 

connection_path= 'postgresql://postgres:'+ credentials_fd.password+'@localhost:5432/postgres'
engine = sa.create_engine(connection_path)

%reload_ext sql
%sql $engine.url

In [2]:
sql_query = pd.read_sql_query ('''
                               
with status AS 
						(
							select 
							customer,
							status,
							ROW_NUMBER() OVER(partition by customer order by created desc) rn 
							from subscriptions
						)
,latest_transaction as 
						(
							select
							customer,
							created,
							ROW_NUMBER() OVER(partition by customer order by created  desc) rn
							from 
							charges
						)
,total_charges as 
						(
							select
							customer,
							sum(amount) sum_amount
							from 
							charges
							group by 1 
						)




SELECT
	customers.customer,
	case when customers.total_report_orders='NULL' then 0 else customers.total_report_orders::INT end as frequency,
	status.status,
	current_date-coalesce(latest_transaction.created::date,'2020-01-01'::date) as recency,
	coalesce(total_charges.sum_amount,0) as monetary
from 
customers 
left join status  on status.customer = customers.customer and status.rn=1
left join latest_transaction on latest_transaction.customer = customers.customer  and latest_transaction.rn=1
left join total_charges on total_charges.customer = customers.customer 

                               ''', engine)

main = pd.DataFrame(sql_query, columns = ['customer','frequency','status','recency','monetary'])

active_cust=main[main['status']=='active']
cancelled_cust=main[main['status']=='canceled']
never_subscribed=main[main['status'].isnull()]

print('Number of subscribers: ', active_cust.shape[0])
print('Number of customers who cancelled their subscriptions: ', cancelled_cust.shape[0])
print('Number of customers who have never subscribed: ', never_subscribed.shape[0])

Number of subscribers:  481
Number of customers who cancelled their subscriptions:  68
Number of customers who have never subscribed:  1486


#### RFM on Subscribers

In [3]:
active_cust.head()

,customer,frequency,status,recency,monetary
2,00140bcecb851d2fb8490f5bc49d5e5a,78,active,220,77500
6,006270c7072bc2da300b12790cf909d7,314,active,220,302500
7,008da0e53a253505d261d0428e0216e9,13,active,238,15000
9,00f54147431e6f0bffec4252ab8f0862,0,active,222,5000
27,0211e7ee3701f70dff75b351a5c5cd61,468,active,220,466500


In [16]:
# Rank each metric R , F & M
active_cust['R_rank'] = active_cust['recency'].rank( ascending=False)
active_cust['F_rank'] = active_cust['frequency'].rank(ascending=True)
active_cust['M_rank'] = active_cust['monetary'].rank(ascending=True)

# normalize each rank with Max rank
active_cust['R_rank_norm']=(active_cust['R_rank']/active_cust['R_rank'].max())*100
active_cust['F_rank_norm']=(active_cust['F_rank']/active_cust['F_rank'].max())*100
active_cust['M_rank_norm']=(active_cust['F_rank']/active_cust['M_rank'].max())*100

# Now apply our equation and create final score **Consolidated Score = 0.15*R_rank_norm + 0.28*F_rank_norm + 0.57M_rank_norm**
active_cust['RFM_Score']=0.15*active_cust['R_rank_norm']+0.28*active_cust['F_rank_norm']+0.57*active_cust['M_rank_norm']
active_cust=active_cust.round(0)
active_cust.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
2,00140bcecb851d2fb8490f5bc49d5e5a,78,active,220,77500,402.0,310.0,287.0,100.0,65.0,65.0,70.0,Medium Value Customer
6,006270c7072bc2da300b12790cf909d7,314,active,220,302500,402.0,425.0,420.0,100.0,88.0,88.0,90.0,High Value Customer
7,008da0e53a253505d261d0428e0216e9,13,active,238,15000,93.0,140.0,134.0,23.0,29.0,29.0,28.0,Low value Customer
9,00f54147431e6f0bffec4252ab8f0862,0,active,222,5000,290.0,12.0,68.0,72.0,2.0,2.0,13.0,Low value Customer
27,0211e7ee3701f70dff75b351a5c5cd61,468,active,220,466500,402.0,446.0,445.0,100.0,93.0,93.0,94.0,High Value Customer


In [5]:
## Segment customers based on RFM score
# 0 - 50 - Low valued customer
# 50 - 75 - Medium valued customer
# 76 - 100 - High valued customer
active_cust["Customer_segment"]=np.where(active_cust['RFM_Score'] > 75 ,"High Value Customer",(np.where(active_cust['RFM_Score'] < 50 , "Low value Customer" ,"Medium Value Customer")))
active_cust.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
2,00140bcecb851d2fb8490f5bc49d5e5a,78,active,220,77500,402.0,310.0,287.0,100.0,65.0,65.0,70.0,Medium Value Customer
6,006270c7072bc2da300b12790cf909d7,314,active,220,302500,402.0,425.0,420.0,100.0,88.0,88.0,90.0,High Value Customer
7,008da0e53a253505d261d0428e0216e9,13,active,238,15000,93.0,140.0,134.0,23.0,29.0,29.0,28.0,Low value Customer
9,00f54147431e6f0bffec4252ab8f0862,0,active,222,5000,290.0,12.0,68.0,72.0,2.0,2.0,13.0,Low value Customer
27,0211e7ee3701f70dff75b351a5c5cd61,468,active,220,466500,402.0,446.0,445.0,100.0,93.0,93.0,94.0,High Value Customer


In [6]:
active_cust['Customer_segment'].value_counts()

Low value Customer       228
High Value Customer      128
Medium Value Customer    125
Name: Customer_segment, dtype: int64

Turns out we have 228 low value customers with an active subscription. Those who are in the lower end of this segment are most likely to churn. We can do targeted promotion for such subscribers.

#### RFM on customers who cancelled their subscription

In [7]:
cancelled_cust.head()

,customer,frequency,status,recency,monetary
21,01b83fa9da336e00bdf7f4ae57569c51,4,canceled,377,3000
68,062f1f73598da7ec8da125a61f0967f6,6,canceled,344,5000
135,0cc6423f1a21a843cd5bf1451827f025,73,canceled,226,67000
199,128f996933334160bcf2d33567a53396,91,canceled,223,102500
227,16d0b30f1123e2d84b39286968a3e1ca,4,canceled,412,8500


In [17]:
# Rank each metric R , F & M
cancelled_cust['R_rank'] = cancelled_cust['recency'].rank( ascending=False)
cancelled_cust['F_rank'] = cancelled_cust['frequency'].rank(ascending=True)
cancelled_cust['M_rank'] = cancelled_cust['monetary'].rank(ascending=True)

# normalize each rank with Max rank
cancelled_cust['R_rank_norm']=(cancelled_cust['R_rank']/cancelled_cust['R_rank'].max())*100
cancelled_cust['F_rank_norm']=(cancelled_cust['F_rank']/cancelled_cust['F_rank'].max())*100
cancelled_cust['M_rank_norm']=(cancelled_cust['F_rank']/cancelled_cust['M_rank'].max())*100

# Now apply our equation and create final score **Consolidated Score = 0.15*R_rank_norm + 0.28*F_rank_norm + 0.57M_rank_norm**
cancelled_cust['RFM_Score']=0.15*cancelled_cust['R_rank_norm']+0.28*cancelled_cust['F_rank_norm']+0.57*cancelled_cust['M_rank_norm']
cancelled_cust=cancelled_cust.round(0)

cancelled_cust.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
21,01b83fa9da336e00bdf7f4ae57569c51,4,canceled,377,3000,18.0,18.0,18.0,26.0,26.0,26.0,26.0,Low value Customer
68,062f1f73598da7ec8da125a61f0967f6,6,canceled,344,5000,24.0,22.0,22.0,36.0,32.0,32.0,32.0,Low value Customer
135,0cc6423f1a21a843cd5bf1451827f025,73,canceled,226,67000,52.0,55.0,54.0,78.0,81.0,81.0,81.0,High Value Customer
199,128f996933334160bcf2d33567a53396,91,canceled,223,102500,60.0,58.0,58.0,90.0,85.0,85.0,85.0,High Value Customer
227,16d0b30f1123e2d84b39286968a3e1ca,4,canceled,412,8500,13.0,18.0,28.0,19.0,26.0,26.0,25.0,Low value Customer


In [9]:
## Segment customers based on RFM score
# 0 - 50 - Low valued customer
# 50 - 75 - Medium valued customer
# 76 - 100 - High valued customer
cancelled_cust["Customer_segment"]=np.where(cancelled_cust['RFM_Score'] > 75 ,"High Value Customer",(np.where(cancelled_cust['RFM_Score'] < 50 , "Low value Customer" ,"Medium Value Customer")))
cancelled_cust.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
21,01b83fa9da336e00bdf7f4ae57569c51,4,canceled,377,3000,18.0,18.0,18.0,26.0,26.0,26.0,26.0,Low value Customer
68,062f1f73598da7ec8da125a61f0967f6,6,canceled,344,5000,24.0,22.0,22.0,36.0,32.0,32.0,32.0,Low value Customer
135,0cc6423f1a21a843cd5bf1451827f025,73,canceled,226,67000,52.0,55.0,54.0,78.0,81.0,81.0,81.0,High Value Customer
199,128f996933334160bcf2d33567a53396,91,canceled,223,102500,60.0,58.0,58.0,90.0,85.0,85.0,85.0,High Value Customer
227,16d0b30f1123e2d84b39286968a3e1ca,4,canceled,412,8500,13.0,18.0,28.0,19.0,26.0,26.0,25.0,Low value Customer


In [10]:
cancelled_cust['Customer_segment'].value_counts()

Low value Customer       33
High Value Customer      18
Medium Value Customer    17
Name: Customer_segment, dtype: int64

18 high value customers with churned earlier. We can target them to sign up for our plans back again.

#### RFM on customers who never subscribed

In [11]:
never_subscribed.head()

,customer,frequency,status,recency,monetary
0,000812cc5dc776a110d375073caecb67,30,None,570,29500
1,0010af551f08abaed739af4f68a49b48,2,None,548,1500
3,0019180d3d1dc53f858e8e89966b1906,9,None,434,8000
4,0022d1ebfb894d6f07841b12e7f23151,19,None,228,25500
5,005b92d276c538ba86d3270a8c49a710,58,None,223,35500


In [18]:
# Rank each metric R , F & M
never_subscribed['R_rank'] = never_subscribed['recency'].rank( ascending=False)
never_subscribed['F_rank'] = never_subscribed['frequency'].rank(ascending=True)
never_subscribed['M_rank'] = never_subscribed['monetary'].rank(ascending=True)

# normalize each rank with Max rank
never_subscribed['R_rank_norm']=(never_subscribed['R_rank']/never_subscribed['R_rank'].max())*100
never_subscribed['F_rank_norm']=(never_subscribed['F_rank']/never_subscribed['F_rank'].max())*100
never_subscribed['M_rank_norm']=(never_subscribed['F_rank']/never_subscribed['M_rank'].max())*100

# Now apply our equation and create final score **Consolidated Score = 0.15*R_rank_norm + 0.28*F_rank_norm + 0.57M_rank_norm**
never_subscribed['RFM_Score']=0.15*never_subscribed['R_rank_norm']+0.28*never_subscribed['F_rank_norm']+0.57*never_subscribed['M_rank_norm']
never_subscribed=never_subscribed.round(0)

never_subscribed.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
0,000812cc5dc776a110d375073caecb67,30,None,570,29500,266.0,456.0,462.0,19.0,31.0,31.0,29.0,Low value Customer
1,0010af551f08abaed739af4f68a49b48,2,None,548,1500,284.0,172.0,186.0,21.0,12.0,12.0,13.0,Low value Customer
3,0019180d3d1dc53f858e8e89966b1906,9,None,434,8000,393.0,280.0,280.0,28.0,19.0,19.0,20.0,Low value Customer
4,0022d1ebfb894d6f07841b12e7f23151,19,None,228,25500,941.0,360.0,422.0,68.0,24.0,24.0,31.0,Low value Customer
5,005b92d276c538ba86d3270a8c49a710,58,None,223,35500,1170.0,672.0,522.0,85.0,45.0,45.0,51.0,Medium Value Customer


In [13]:
## Segment customers based on RFM score
# 0 - 50 - Low valued customer
# 50 - 75 - Medium valued customer
# 76 - 100 - High valued customer
never_subscribed["Customer_segment"]=np.where(never_subscribed['RFM_Score'] > 75 ,"High Value Customer",(np.where(never_subscribed['RFM_Score'] < 50 , "Low value Customer" ,"Medium Value Customer")))
never_subscribed.head()

,customer,frequency,status,recency,monetary,R_rank,F_rank,M_rank,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
0,000812cc5dc776a110d375073caecb67,30,None,570,29500,266.0,456.0,462.0,19.0,31.0,31.0,29.0,Low value Customer
1,0010af551f08abaed739af4f68a49b48,2,None,548,1500,284.0,172.0,186.0,21.0,12.0,12.0,13.0,Low value Customer
3,0019180d3d1dc53f858e8e89966b1906,9,None,434,8000,393.0,280.0,280.0,28.0,19.0,19.0,20.0,Low value Customer
4,0022d1ebfb894d6f07841b12e7f23151,19,None,228,25500,941.0,360.0,422.0,68.0,24.0,24.0,31.0,Low value Customer
5,005b92d276c538ba86d3270a8c49a710,58,None,223,35500,1170.0,672.0,522.0,85.0,45.0,45.0,51.0,Medium Value Customer


In [14]:
never_subscribed['Customer_segment'].value_counts()

Low value Customer       718
Medium Value Customer    413
High Value Customer      355
Name: Customer_segment, dtype: int64

355 high value customers who might be targeted for new subscriptions.

In [15]:
%%html
<div class='tableauPlaceholder' id='viz1669965567927' style='position: relative'><noscript><a href='#'><img alt='Retention Model ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ro&#47;Roofr-RetentionAnalysis&#47;RetentionModel&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Roofr-RetentionAnalysis&#47;RetentionModel' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ro&#47;Roofr-RetentionAnalysis&#47;RetentionModel&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1669965567927');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>